# Exploration du corpus de documents

Ce notebook explore le corpus de documents et prépare les données pour le traitement.

In [ ]:
import sys
sys.path.append('..')

from src.preprocessing.document_loader import DocumentLoader
from src.preprocessing.text_cleaner import TextCleaner
from pathlib import Path
import json

## 1. Chargement des documents

In [ ]:
# Configuration
raw_dir = Path("../data/raw")
output_file = Path("../data/processed/documents.json")

# Charger les documents
print("Chargement des documents...")
loader = DocumentLoader(raw_dir)
documents = loader.load_all_documents()

print(f"Nombre de documents chargés: {len(documents)}")

## 2. Analyse des documents

In [ ]:
# Statistiques des documents
formats = {}
total_chars = 0

for doc in documents:
    fmt = doc['metadata']['format']
    formats[fmt] = formats.get(fmt, 0) + 1
    total_chars += len(doc['text'])

print("Répartition des formats:")
for fmt, count in formats.items():
    print(f"  {fmt}: {count} documents")

print(f"\nNombre total de caractères: {total_chars:,}")
print(f"Longueur moyenne par document: {total_chars/len(documents):,.0f} caractères")

## 3. Nettoyage des textes

In [ ]:
# Nettoyer les textes
print("Nettoyage des textes...")
cleaner = TextCleaner()

for doc in documents:
    original_length = len(doc['text'])
    doc['text'] = cleaner.clean(doc['text'])
    cleaned_length = len(doc['text'])
    doc['paragraphs'] = cleaner.split_into_paragraphs(doc['text'])
    
    # Statistiques de nettoyage
    if original_length > 0:
        reduction = (original_length - cleaned_length) / original_length * 100
        doc['cleaning_stats'] = {
            'original_length': original_length,
            'cleaned_length': cleaned_length,
            'reduction_percent': reduction
        }

## 4. Sauvegarde des documents traités

In [ ]:
# Sauvegarder
print(f"Sauvegarde de {len(documents)} documents...")
output_file.parent.mkdir(parents=True, exist_ok=True)
loader.save_processed(documents, output_file)

print("Terminé!")

## 5. Visualisation des données

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Créer un DataFrame pour l'analyse
df_data = []
for doc in documents:
    df_data.append({
        'filename': doc['filename'],
        'format': doc['metadata']['format'],
        'size': doc['metadata']['size'],
        'text_length': len(doc['text']),
        'paragraphs': len(doc['paragraphs'])
    })

df = pd.DataFrame(df_data)

# Visualisations
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Distribution des formats
df['format'].value_counts().plot(kind='bar', ax=axes[0, 0], title='Distribution des formats')

# Distribution des tailles de texte
df['text_length'].hist(bins=30, ax=axes[0, 1], title='Distribution des longueurs de texte')

# Relation taille vs nombre de paragraphes
axes[1, 0].scatter(df['text_length'], df['paragraphs'], alpha=0.5)
axes[1, 0].set_xlabel('Longueur du texte')
axes[1, 0].set_ylabel('Nombre de paragraphes')
axes[1, 0].set_title('Relation texte/paragraphes')

# Boxplot par format
df.boxplot(column='text_length', by='format', ax=axes[1, 1])
axes[1, 1].set_title('Longueur de texte par format')
axes[1, 1].set_ylabel('Longueur')

plt.tight_layout()
plt.show()